In [2]:
%load_ext jupyternotify

<IPython.core.display.Javascript object>

In [3]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV


import os
import glob
import csv

import pandas as pd
from sklearn.model_selection import KFold, StratifiedKFold

from keras.applications import mobilenet
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras import layers, models
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

import numpy as np
import tensorflow as tf

from csv import reader

import itertools

# libraries for outputting camera images
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import clear_output

import pickle

In [8]:
def create_new_model(img_size, n_cats, learning_rate):
    columns = [str(i) for i in range(n_cats)]
    base_model = mobilenet.MobileNet(weights='imagenet', 
                                     include_top=False, 
                                     input_shape=(img_size, img_size, 3))
    for layer in base_model.layers:
        layer.trainable = False
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dense(len(columns), activation='softmax')(x)
    model = models.Model(inputs=base_model.input, outputs=x)
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    
    return model

def CrossValid(
    tuning_pars, n_inst=424, n_cats=5, num_epochs=10, n_splits=10, img_size=224, imgAugment = True
):
    
    if n_inst == 871:
        csv_filename = f'data/train_cv_224/handOverFace_n871_{n_cats}cats_1.csv'
    else:
        csv_filename = f'data/train_cv_224/training_labels_handOverFace_n424_{n_cats}cats.csv'
    
    targetDir = 'data/train_cv_224'
    train_data = pd.read_csv(csv_filename, sep=',', header=0)
    
    idg_valid = ImageDataGenerator(preprocessing_function=mobilenet.preprocess_input)
    if imgAugment == True:
        idg_train = ImageDataGenerator(
            preprocessing_function=mobilenet.preprocess_input, width_shift_range=[-25, 25], 
            height_shift_range=[-25, 25], horizontal_flip=True, brightness_range=[0.6,1.3],
            channel_shift_range=15
        )
    else:
        idg_train = idg_valid
        
    img_size = 224
    columns = [str(i) for i in range(n_cats)]
    
    # settuping the kfolds
    kf = KFold(n_splits=n_splits, shuffle=True)    
    
    num_samples = train_data.shape[0]
    
    # getting all possible combinations of hyperparameters
    keys, values = zip(*tuning_pars.items())
    permutations_dicts = [dict(zip(keys, v)) for v in itertools.product(*values)]
    
    # for each combination of parameters
    for permut in permutations_dicts:
        
        dir_name = f"runningInfo_habitBreaker_imgAug{imgAugment}_n{n_inst}" \
                   f"_ncats{n_cats}_batchSize{permut['batch_size']}_learnRate{permut['learning_rate']}"
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)
        
        valid_accuracy = []
        valid_loss = []
        train_accuracy = []
        train_loss = []
        
        print(f"Dealing with permut: {permut}")
        k_idx = 0
        
        # for each k-th training-validation set
        for train_index, val_index in list(kf.split(np.zeros(num_samples), train_data)):
            
            k_idx += 1
                
            training_data = train_data.iloc[train_index]
            validation_data = train_data.iloc[val_index]

            # one hot encoding because it is softmax    
            train_data_generator = idg_train.flow_from_dataframe(dataframe=training_data, directory=targetDir,
                                                          x_col="filename", 
                                                           y_col=columns,
                                                          shuffle=True,
                                                          target_size=(img_size, img_size), 
                                                           batch_size=permut['batch_size'], 
                                                          class_mode='raw')
            valid_data_generator = idg_valid.flow_from_dataframe(dataframe=validation_data, directory=targetDir,
                                                          x_col="filename", 
                                                           y_col=columns,
                                                          shuffle=True,
                                                          target_size=(img_size, img_size), 
                                                           batch_size=permut['batch_size'],
                                                          class_mode='raw')

            model = create_new_model(img_size=224, n_cats=n_cats, learning_rate=permut['learning_rate'])

            step_size = train_data_generator.n//train_data_generator.batch_size
            step_size_valid = valid_data_generator.n//valid_data_generator.batch_size

            with tf.device('/device:GPU:0'):
                history = model.fit(train_data_generator, epochs=num_epochs, steps_per_epoch=step_size, 
                                validation_data=valid_data_generator,
                               validation_steps=step_size_valid,
                                callbacks=[])

            results = model.evaluate(valid_data_generator)
            results = dict(zip(model.metrics_names, results))
            
            output_dir_name = os.path.join(dir_name, f"history_kIdx{k_idx}.pickle")
            with open(output_dir_name, "wb") as f:
                pickle.dump(history.history, f)

            valid_accuracy.append(results['accuracy'])
            valid_loss.append(results['loss'])
            train_accuracy.append(history.history['accuracy'][-1])
            train_loss.append(history.history['loss'][-1])
        
        # saving results of one permut
        output_dir_name = os.path.join(dir_name, "validationResults.pickle")
        with open(output_dir_name, "wb") as f:
            pickle.dump({'val_acc':valid_accuracy, 'val_loss':valid_loss, 'acc':train_accuracy, 'loss':train_loss}, f)

In [9]:
%%notify
# change to only run the 10th epoch
tuning_pars = {'batch_size': [32, 68, 128], 'learning_rate':[1e-3, 1e-4]}
CrossValid(tuning_pars, n_inst=871, n_cats=5, num_epochs=10, n_splits=10, imgAugment=False)

Dealing with permut: {'batch_size': 32, 'learning_rate': 0.001}
Found 3919 validated image filenames.
Found 436 validated image filenames.
Epoch 1/10
122/122 [==============================] - 52s 392ms/step - loss: 1.0846 - accuracy: 0.6358 - val_loss: 0.8720 - val_accuracy: 0.7067
Epoch 2/10
122/122 [==============================] - 46s 377ms/step - loss: 0.3669 - accuracy: 0.8737 - val_loss: 0.7486 - val_accuracy: 0.7668
Epoch 3/10
122/122 [==============================] - 47s 384ms/step - loss: 0.2321 - accuracy: 0.9149 - val_loss: 0.6816 - val_accuracy: 0.7957
Epoch 4/10
122/122 [==============================] - 47s 386ms/step - loss: 0.1985 - accuracy: 0.9254 - val_loss: 0.7384 - val_accuracy: 0.7837
Epoch 5/10
122/122 [==============================] - 47s 389ms/step - loss: 0.1075 - accuracy: 0.9639 - val_loss: 0.5261 - val_accuracy: 0.8389
Epoch 6/10
122/122 [==============================] - 46s 381ms/step - loss: 0.0851 - accuracy: 0.9712 - val_loss: 0.5045 - val_accuracy

Epoch 1/10
122/122 [==============================] - 61s 457ms/step - loss: 1.0657 - accuracy: 0.6428 - val_loss: 0.9606 - val_accuracy: 0.6274
Epoch 2/10
122/122 [==============================] - 52s 429ms/step - loss: 0.3764 - accuracy: 0.8690 - val_loss: 0.7957 - val_accuracy: 0.7620
Epoch 3/10
122/122 [==============================] - 50s 412ms/step - loss: 0.2642 - accuracy: 0.9020 - val_loss: 0.8930 - val_accuracy: 0.7452
Epoch 4/10
122/122 [==============================] - 52s 428ms/step - loss: 0.1809 - accuracy: 0.9415 - val_loss: 0.5894 - val_accuracy: 0.8245
Epoch 5/10
122/122 [==============================] - 52s 427ms/step - loss: 0.1213 - accuracy: 0.9557 - val_loss: 0.6590 - val_accuracy: 0.8365
Epoch 6/10
122/122 [==============================] - 52s 430ms/step - loss: 0.0911 - accuracy: 0.9702 - val_loss: 0.7595 - val_accuracy: 0.8149
Epoch 7/10
122/122 [==============================] - 49s 404ms/step - loss: 0.0793 - accuracy: 0.9736 - val_loss: 0.7370 - val_ac

122/122 [==============================] - 54s 404ms/step - loss: 1.2091 - accuracy: 0.5313 - val_loss: 0.7924 - val_accuracy: 0.7692
Epoch 2/10
122/122 [==============================] - 48s 396ms/step - loss: 0.4183 - accuracy: 0.8676 - val_loss: 0.6459 - val_accuracy: 0.7812
Epoch 3/10
122/122 [==============================] - 49s 398ms/step - loss: 0.2492 - accuracy: 0.9430 - val_loss: 0.5761 - val_accuracy: 0.8077
Epoch 4/10
122/122 [==============================] - 48s 396ms/step - loss: 0.1702 - accuracy: 0.9641 - val_loss: 0.5508 - val_accuracy: 0.8269
Epoch 5/10
122/122 [==============================] - 49s 405ms/step - loss: 0.1103 - accuracy: 0.9839 - val_loss: 0.5492 - val_accuracy: 0.8341
Epoch 6/10
122/122 [==============================] - 49s 400ms/step - loss: 0.0774 - accuracy: 0.9916 - val_loss: 0.5983 - val_accuracy: 0.8438
Epoch 7/10
122/122 [==============================] - 48s 395ms/step - loss: 0.0562 - accuracy: 0.9956 - val_loss: 0.5662 - val_accuracy: 0.8

Epoch 2/10
122/122 [==============================] - 47s 387ms/step - loss: 0.4316 - accuracy: 0.8623 - val_loss: 0.5826 - val_accuracy: 0.8005
Epoch 3/10
122/122 [==============================] - 48s 391ms/step - loss: 0.2555 - accuracy: 0.9435 - val_loss: 0.5400 - val_accuracy: 0.8149
Epoch 4/10
122/122 [==============================] - 48s 398ms/step - loss: 0.1663 - accuracy: 0.9690 - val_loss: 0.4948 - val_accuracy: 0.8221
Epoch 5/10
122/122 [==============================] - 48s 390ms/step - loss: 0.1074 - accuracy: 0.9841 - val_loss: 0.4998 - val_accuracy: 0.8149
Epoch 6/10
122/122 [==============================] - 47s 387ms/step - loss: 0.0805 - accuracy: 0.9910 - val_loss: 0.5114 - val_accuracy: 0.8125
Epoch 7/10
122/122 [==============================] - 48s 390ms/step - loss: 0.0607 - accuracy: 0.9914 - val_loss: 0.5048 - val_accuracy: 0.8389
Epoch 8/10
122/122 [==============================] - 47s 387ms/step - loss: 0.0433 - accuracy: 0.9971 - val_loss: 0.5063 - val_ac

57/57 [==============================] - 49s 853ms/step - loss: 0.3201 - accuracy: 0.8902 - val_loss: 0.6676 - val_accuracy: 0.7181
Epoch 3/10
57/57 [==============================] - 50s 883ms/step - loss: 0.1844 - accuracy: 0.9378 - val_loss: 0.4463 - val_accuracy: 0.8431
Epoch 4/10
57/57 [==============================] - 49s 865ms/step - loss: 0.1179 - accuracy: 0.9649 - val_loss: 0.4404 - val_accuracy: 0.8480
Epoch 5/10
57/57 [==============================] - 49s 853ms/step - loss: 0.0716 - accuracy: 0.9819 - val_loss: 0.5875 - val_accuracy: 0.8015
Epoch 6/10
57/57 [==============================] - 49s 851ms/step - loss: 0.0769 - accuracy: 0.9794 - val_loss: 0.5369 - val_accuracy: 0.8505
Epoch 7/10
57/57 [==============================] - 49s 861ms/step - loss: 0.0350 - accuracy: 0.9924 - val_loss: 0.3998 - val_accuracy: 0.8799
Epoch 8/10
57/57 [==============================] - 49s 857ms/step - loss: 0.0266 - accuracy: 0.9933 - val_loss: 0.4546 - val_accuracy: 0.8750
Epoch 9/10

...

Epoch 9/10
30/30 [==============================] - 59s 2s/step - loss: 0.0178 - accuracy: 0.9969 - val_loss: 0.6892 - val_accuracy: 0.8281
Epoch 10/10
4/4 [==============================] - 7s 1s/step - loss: 0.6138 - accuracy: 0.8578
Found 3919 validated image filenames.
Found 436 validated image filenames.
Epoch 1/10
30/30 [==============================] - 68s 2s/step - loss: 1.1265 - accuracy: 0.6004 - val_loss: 0.8097 - val_accuracy: 0.7057
Epoch 2/10
30/30 [==============================] - 59s 2s/step - loss: 0.2953 - accuracy: 0.8975 - val_loss: 0.7968 - val_accuracy: 0.6719
Epoch 3/10
30/30 [==============================] - 65s 2s/step - loss: 0.1256 - accuracy: 0.9634 - val_loss: 0.6569 - val_accuracy: 0.7578
Epoch 4/10
30/30 [==============================] - 60s 2s/step - loss: 0.0789 - accuracy: 0.9788 - val_loss: 0.5494 - val_accuracy: 0.7969
Epoch 5/10
30/30 [==============================] - 66s 2s/step - loss: 0.0511 - accuracy: 0.9870 - val_loss: 0.5504 - val_accura

Epoch 1/10
30/30 [==============================] - 59s 2s/step - loss: 1.0104 - accuracy: 0.6251 - val_loss: 0.8281 - val_accuracy: 0.6484
Epoch 2/10
30/30 [==============================] - 52s 2s/step - loss: 0.2863 - accuracy: 0.9037 - val_loss: 0.6669 - val_accuracy: 0.7552
Epoch 3/10
30/30 [==============================] - 52s 2s/step - loss: 0.1639 - accuracy: 0.9470 - val_loss: 0.5235 - val_accuracy: 0.8281
Epoch 4/10
30/30 [==============================] - 53s 2s/step - loss: 0.0808 - accuracy: 0.9756 - val_loss: 0.5052 - val_accuracy: 0.8125
Epoch 5/10
30/30 [==============================] - 52s 2s/step - loss: 0.0588 - accuracy: 0.9848 - val_loss: 0.4858 - val_accuracy: 0.8359
Epoch 6/10
30/30 [==============================] - 53s 2s/step - loss: 0.0409 - accuracy: 0.9890 - val_loss: 0.4742 - val_accuracy: 0.8568
Epoch 7/10
30/30 [==============================] - 53s 2s/step - loss: 0.0289 - accuracy: 0.9931 - val_loss: 0.4555 - val_accuracy: 0.8568
Epoch 8/10
30/30 [==

30/30 [==============================] - 52s 2s/step - loss: 0.4040 - accuracy: 0.8891 - val_loss: 0.7364 - val_accuracy: 0.7812
Epoch 4/10
30/30 [==============================] - 53s 2s/step - loss: 0.2783 - accuracy: 0.9438 - val_loss: 0.6535 - val_accuracy: 0.7969
Epoch 5/10
30/30 [==============================] - 53s 2s/step - loss: 0.2062 - accuracy: 0.9708 - val_loss: 0.5964 - val_accuracy: 0.8125
Epoch 6/10
30/30 [==============================] - 52s 2s/step - loss: 0.1533 - accuracy: 0.9836 - val_loss: 0.5585 - val_accuracy: 0.8125
Epoch 7/10
30/30 [==============================] - 53s 2s/step - loss: 0.1189 - accuracy: 0.9886 - val_loss: 0.5046 - val_accuracy: 0.8307
Epoch 8/10
30/30 [==============================] - 52s 2s/step - loss: 0.0955 - accuracy: 0.9944 - val_loss: 0.5054 - val_accuracy: 0.8255
Epoch 9/10
30/30 [==============================] - 52s 2s/step - loss: 0.0826 - accuracy: 0.9950 - val_loss: 0.4618 - val_accuracy: 0.8411
Epoch 10/10
4/4 [==============

Epoch 6/10
30/30 [==============================] - 52s 2s/step - loss: 0.1459 - accuracy: 0.9841 - val_loss: 0.5820 - val_accuracy: 0.8177
Epoch 7/10
30/30 [==============================] - 51s 2s/step - loss: 0.1140 - accuracy: 0.9926 - val_loss: 0.5727 - val_accuracy: 0.8151
Epoch 8/10
30/30 [==============================] - 52s 2s/step - loss: 0.0916 - accuracy: 0.9953 - val_loss: 0.5538 - val_accuracy: 0.8151
Epoch 9/10
30/30 [==============================] - 53s 2s/step - loss: 0.0734 - accuracy: 0.9983 - val_loss: 0.5467 - val_accuracy: 0.8151
Epoch 10/10
4/4 [==============================] - 5s 1s/step - loss: 0.5232 - accuracy: 0.8299
Found 3920 validated image filenames.
Found 435 validated image filenames.
Epoch 1/10
30/30 [==============================] - 60s 2s/step - loss: 1.3756 - accuracy: 0.4334 - val_loss: 1.2011 - val_accuracy: 0.5286
Epoch 2/10
30/30 [==============================] - 51s 2s/step - loss: 0.5943 - accuracy: 0.8063 - val_loss: 0.9606 - val_accura

<IPython.core.display.Javascript object>